### Data Merge

##### Declarations

In [22]:
#Importing utilities file with library imports and helper functions
%run "utils.ipynb"

In [23]:
#Setting pandas option to disable scientific notation
pd.set_option('display.max_rows', 10000)

In [24]:
#Input and output paths for all pickle files. To be used to load and save these files
#Output
input_path="C:\\Users\\Anuvrat\\OneDrive\\Documents\\milestone1\\data\\inputs"
output_path="C:\\Users\\Anuvrat\\OneDrive\\Documents\\milestone1\\data\\inputs"

In [25]:
#Loading input pickle files
df_income_1901_edited=pd.read_pickle(input_path+"\\df_income_1901_edited.pkl")
df_health_outcomes_edited=pd.read_pickle(input_path+"\\df_health_outcomes_edited.pkl")
df_sdoh_edited=pd.read_pickle(input_path+"\\df_sdoh_edited.pkl")
df_state_politics_edited=pd.read_pickle(input_path+"\\df_state_politics_edited.pkl")
df_state_public_spend_edited=pd.read_pickle(input_path+"\\df_state_public_spend_edited.pkl")
df_acs_edited=pd.read_pickle(input_path+"\\df_acs_edited.pkl")
df_zip2st_edited=pd.read_pickle(input_path+"\\df_zip2st_edited.pkl")

In [26]:
#Printing shapes to check which file needs most attention for data compression
for df in [df_income_1901_edited, df_health_outcomes_edited, df_sdoh_edited, df_state_politics_edited, df_state_public_spend_edited, df_acs_edited]:
    print(df.shape)

(33774, 15)
(1169992, 10)
(291024, 4)
(57, 3)
(255, 3)
(33120, 14)


##### ZIP level data

In [27]:
df_income_1901_edited.ZIP.nunique(), df_health_outcomes_edited.ZIP.nunique(), df_sdoh_edited.ZIP.nunique(), df_acs_edited.ZIP.nunique()

(33774, 32409, 32336, 33120)

In [28]:
df_income_1901_essential=df_income_1901_edited[['ZIP', 'Estimate Households Total', 'Percent Households lt 10k', 
                                                'Estimate Households Median income (dollars)', 'Estimate Families Total',
                                                'Estimate Families Median income (dollars)', 'Estimate Nonfamily households Total',
                                                'Estimate Nonfamily households Median income (dollars)', 'Percent Households that are Families']]
df_income_1901_essential.shape

(33774, 9)

In [29]:
df_health_outcomes_essential=df_health_outcomes_edited[['ZIP', 'TotalPopulation', 'Short_Question_Text', 'Data_Value']]
df_health_outcomes_essential_wide=df_health_outcomes_essential.pivot(index=['ZIP', 'TotalPopulation'], columns='Short_Question_Text', 
                                                                     values='Data_Value').reset_index().rename_axis(None, axis=1)
#display(df_health_outcomes_essential_wide.head())
df_health_outcomes_essential_wide.shape

(32409, 39)

In [30]:
df_sdoh_essential_wide=df_sdoh_edited.pivot(index=['ZIP', 'TotalPopulation'], columns='Measure', 
                                                                     values='Data_Value').reset_index().rename_axis(None, axis=1)
#display(df_sdoh_essential_wide.head())
df_sdoh_essential_wide.shape

(32336, 11)

##### State level data

In [31]:
#Checking if any of the state level data needs to be pivoted
print("df_state_politics_edited", df_state_politics_edited.shape, df_state_politics_edited.State.nunique())
print("df_zip2st_edited", df_zip2st_edited.shape, df_zip2st_edited.ZIP.nunique())
print("df_state_public_spend_edited", df_state_public_spend_edited.shape, df_state_public_spend_edited['State Name'].nunique())

df_state_politics_edited (57, 3) 57
df_zip2st_edited (39368, 2) 39368
df_state_public_spend_edited (255, 3) 51


In [32]:
#Turns out state_public_spend_edited will need to be pivoted as it has multiple values of year for each state
df_state_public_spend_essential_wide=df_state_public_spend_edited.pivot(
    index=['State Name'], columns='TimeFrame', values='Data').reset_index().rename_axis(
        None, axis=1).add_prefix('spend_').rename(columns={'spend_State Name':'State Name'})
df_state_public_spend_essential_wide.head()

,State Name,spend_2017,spend_2018,spend_2019,spend_2020,spend_2021
0,Alabama,56.43214,57.23,54.25,46.81,52.35
1,Alaska,114.43666,96.52,98.25,215.15,90.17
2,Arizona,8.83289,9.66,15.25,24.34,15.32
3,Arkansas,52.29329,50.57,48.23,45.90,42.94
4,California,61.69589,65.71,71.89,69.87,78.47


In [33]:
#Geographic information (Lat-Long)
geocoords=df_health_outcomes_edited[['ZIP', 'longitude', 'latitude']].drop_duplicates()
geocoords.shape, geocoords.ZIP.nunique()

((32409, 3), 32409)

##### Creating combined dataset

In [34]:
#Combined_dataset_zip contain ZIP level features. Combined_dataset_state contains State level features. Combined_dataset contains all features
combined_dataset_zip=pd.merge(pd.merge(pd.merge(df_income_1901_essential, df_health_outcomes_essential_wide, on="ZIP", how="outer"), 
         df_sdoh_essential_wide, on="ZIP", how="outer"), df_acs_edited, on="ZIP", how="outer")
combined_dataset_state=pd.merge(pd.merge(df_state_politics_edited, df_state_public_spend_essential_wide, on="State Name", how="outer"),
         df_zip2st_edited, on="State", how="outer")
combined_dataset=pd.merge(pd.merge(combined_dataset_zip, combined_dataset_state, on="ZIP", how="outer"), geocoords, on="ZIP", how="left")
combined_dataset.shape

(39503, 80)

In [35]:
combined_dataset.sample(5)

,ZIP,Estimate Households Total,Percent Households lt 10k,Estimate Households Median income (dollars),Estimate Families Total,Estimate Families Median income (dollars),Estimate Nonfamily households Total,Estimate Nonfamily households Median income (dollars),Percent Households that are Families,TotalPopulation_x,...,State Name,State,Political Affiliation (2008-2020 presidential elections),spend_2017,spend_2018,spend_2019,spend_2020,spend_2021,longitude,latitude
719,02878,6782.0,3.0,92444.0,4473.0,119651.0,2309.0,51972.0,0.659540,15780.0,...,Rhode Island,RI,Deep Blue,57.59099,52.96,57.26,59.16,NaN,-71.173715,41.609072
20601,59416,160.0,0.0,97632.0,113.0,86042.0,47.0,144107.0,0.706250,337.0,...,Montana,MT,Deep Red,24.30714,22.56,21.73,18.91,18.33,-111.496220,48.041906
34866,20218,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,District of Columbia,DC,Deep Blue,138.70818,134.96,363.18,364.53,370.56,NaN,NaN
1628,05763,1257.0,0.0,63738.0,837.0,99583.0,420.0,36308.0,0.665871,2747.0,...,Vermont,VT,Deep Blue,56.15900,47.41,49.20,52.61,56.18,-72.999635,43.726665
9879,30215,13074.0,1.4,113381.0,10689.0,127399.0,2385.0,47645.0,0.817577,33665.0,...,Georgia,GA,Light Red,21.27243,22.42,27.35,27.78,30.82,-84.457108,33.388500


In [36]:
combined_dataset['longitude'].isnull().sum()

7094

In [37]:
#Saving combined dataset
combined_dataset.to_pickle(output_path+"\\combined_dataset_anuvrat.pkl")
combined_dataset.to_csv(output_path+"\\combined_dataset_anuvrat.csv")

##### Checking if combined_dataset has values for all ZIPs

In [38]:
# % Missing values for each column. Function defined in helper functions notebook
perc_missing_num(combined_dataset)

Number of records in dataset: 39503
Missing records summary:


,column,number_missing,percent_missing
0,ZIP,3,0.007594
70,State Name,132,0.334152
71,State,132,0.334152
72,Political Affiliation (2008-2020 presidential ...,319,0.807534
73,spend_2017,319,0.807534
75,spend_2019,319,0.807534
74,spend_2018,319,0.807534
76,spend_2020,319,0.807534
77,spend_2021,2399,6.072956
4,Estimate Families Total,5729,14.502696


In [39]:
df_combined_nona=combined_dataset.dropna()

In [40]:
# % Missing values for each column. Function defined in helper functions notebook
perc_missing_num(df_combined_nona)

Number of records in dataset: 24089
Missing records summary:


,column,number_missing,percent_missing
0,ZIP,0,0.0
57,Estimate!!SEX AND AGE!!Total population,0,0.0
56,Unemployment among people 16 years and older i...,0,0.0
55,Single-parent households,0,0.0
54,Persons of racial or ethnic minority status,0,0.0
53,Persons living below 150% of the poverty level,0,0.0
52,Persons aged 65 years or older,0,0.0
51,No high school diploma among adults aged 25 ye...,0,0.0
58,Estimate!!SEX AND AGE!!Total population!!Sex r...,0,0.0
50,No broadband internet subscription among house...,0,0.0


Insights:
<ol>
<li>29 health outcomes features don't have data for 8,068 or 20% ZIPs. Most of these ZIPs should be 0 or low population as per Census procedure documented in the notes for this dataset.</li>
<li>Another 21 features don't have data for at least 7,000 or more ZIPs.</li>
<li>Another 18 features don't have data for at least 5,704 or more ZIPs.</li>
<li>Removing all ZIPs with missing data leaves us with 24,089 ZIPs (61%) out of 39,503 ZIPs in the original combined dataset.</li>
<li>This is a reasonable sample size to make assumptions. Dropping ZIPs with missing data will remove the noise.</li>
<li>Recommed to move ahead with 24K ZIPs for further analysis.</li>
</ol>

##### Saving dataset without any missing information for any ZIP

In [41]:
#Saving dataset without any missing information
df_combined_nona.to_pickle(output_path+"\\df_combined_nona.pkl")

In [42]:
df_combined_nona.shape

(24089, 80)